<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-setting-up-the-code" data-toc-modified-id="Imports-and-setting-up-the-code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and setting up the code</a></span></li><li><span><a href="#Initialize" data-toc-modified-id="Initialize-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initialize</a></span><ul class="toc-item"><li><span><a href="#Global-settings" data-toc-modified-id="Global-settings-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Global settings</a></span></li><li><span><a href="#Init-instruments" data-toc-modified-id="Init-instruments-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Init instruments</a></span></li><li><span><a href="#Default-(instrument)-settings" data-toc-modified-id="Default-(instrument)-settings-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Default (instrument) settings</a></span></li></ul></li><li><span><a href="#Spectroscopy" data-toc-modified-id="Spectroscopy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Spectroscopy</a></span><ul class="toc-item"><li><span><a href="#CW-Readout-spec" data-toc-modified-id="CW-Readout-spec-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>CW Readout spec</a></span></li></ul></li></ul></div>

# Imports and setting up the code

In [1]:
# %gui qt
# %matplotlib qt

import sys
import os
import time
import logging
from importlib import reload

import numpy as np
from matplotlib import pyplot as plt

logger = logging.getLogger('cqed')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.WARNING)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [2]:
import qcodes as qc
import broadbean as bb
from broadbean.plotting import plotter

In [3]:
from pytopo.mplplots import init_nb_plotting; reload(init_nb_plotting)
from pytopo.mplplots.init_nb_plotting import *
from pytopo.mplplots import plots as mplplots

In [47]:
from pytopo.qctools import instruments as instools; reload(instools)
from pytopo.qctools.instruments import create_inst, add2station
from pytopo.qctools.dataset2 import select_experiment
from pytopo.qctools.measurement import MeasurementExt

from pytopo.rf import alazar_detector; reload(alazar_detector)
from pytopo.rf.alazar_detector import AlazarDetector

from plottr import qcodes_dataset; reload(qcodes_dataset)
from plottr.qcodes_dataset import QcodesDatasetSubscriber

from cqed import broadbean; reload(broadbean)
from cqed.broadbean import BroadBeanSequence, BluePrints

from cqed import bb_sequences; reload(bb_sequences)
from cqed.bb_sequences import TriggeredReadoutSequence, TwoToneSSBSequence

# Initialize

## Global settings

In [5]:
SAMPLE = 'wrapgate_transmon_no1'

In [48]:
qc.config['core']['db_location'] = "f:\OneDrive\Setups\LK2\LK2_data\experiments.db"

BroadBeanSequence.chan_map = {
    1 : ['src_I', 'ro_trigger', 'ro_gate'],
    2 : ['src_Q', 'src_gate', None],
}
BroadBeanSequence.chan_settings[1]['Vpp'] = 2.0
BroadBeanSequence.chan_settings[2]['Vpp'] = 2.0

BroadBeanSequence.sweep_wait = None

TriggeredReadoutSequence.chan_map = {
    1 : ['FILL.1', 'ro_trigger', 'ro_gate'],
#     2 : ['FILL.2', 'src_gate', None]
}

qc.dataset.database.initialise_database()

## Init instruments

In [7]:
inst_list = []

# Alazar
from qcodes.instrument_drivers.AlazarTech import utils; reload(utils)
from qcodes.instrument_drivers.AlazarTech import ATS9870; reload(ATS9870)
from qcodes.instrument_drivers.AlazarTech.ATS9870 import AlazarTech_ATS9870
alazar = instools.create_inst(AlazarTech_ATS9870, 'alazar', force_new_instance=True)
inst_list.append(alazar)

# Alazar aquisition controllers
from pytopo.rf import alazar_acquisition; reload(alazar_acquisition)
from pytopo.rf.alazar_acquisition import RawAcqCtl, AvgDemodCtl, AvgIQCtl

raw_acq = instools.create_inst(RawAcqCtl, 'raw_acq', 'alazar', force_new_instance=True)
inst_list.append(raw_acq)

avgdemod_acq = instools.create_inst(AvgDemodCtl, 'avgdemod_acq', 'alazar', force_new_instance=True)
inst_list.append(avgdemod_acq)

avgiq_acq = instools.create_inst(AvgIQCtl, 'avgiq_acq', 'alazar', force_new_instance=True)
inst_list.append(avgiq_acq)

# RF sources
from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A
LO = instools.create_inst(RohdeSchwarz_SGS100A, 'LO', address="TCPIP0::169.254.2.20", force_new_instance=True)
inst_list.append(LO)

RF = instools.create_inst(RohdeSchwarz_SGS100A, 'RF', address="TCPIP0::169.254.144.157", force_new_instance=True)
inst_list.append(RF)

TP = instools.create_inst(RohdeSchwarz_SGS100A, 'TP', address="TCPIP0::169.254.103.129", force_new_instance=True)
inst_list.append(TP)

from qcodes.instrument_drivers.agilent.E8267C import E8267
S1 = instools.create_inst(E8267, 'S1', address='GPIB::29::INSTR', force_new_instance=True)
inst_list.append(S1)


from pytopo.rf.sources import HeterodyneSource
hetsrc = instools.create_inst(HeterodyneSource, 'hetsrc', RF=RF, LO=LO, force_new_instance=True)
inst_list.append(hetsrc)

# AWG and sequences
from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
awg = instools.create_inst(Tektronix_AWG5014, 'awg', address='TCPIP0::169.254.65.64::inst0::INSTR', force_new_instance=True)
inst_list.append(awg)

station = qc.Station(*inst_list)

Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110512, firmware:3.1.19.15-3.50.124.67) in 0.48s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110511, firmware:3.1.19.15-3.50.124.67) in 0.04s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110432, firmware:3.1.19.15-3.50.124.67) in 0.02s
Connected to: Agilent Technologies E8267C (serial:US42340172, firmware:C.03.85) in 0.12s
Connected to: TEKTRONIX AWG5014C (serial:B050346, firmware:SCPI:99.0 FW:4.6.0.7) in 0.05s


## Default (instrument) settings

In [8]:
with alazar.syncing():
    alazar.clock_source('INTERNAL_CLOCK')
    alazar.sample_rate(int(0.25e9))
    alazar.clock_edge('CLOCK_EDGE_RISING')
    alazar.external_sample_rate(int(1e9))
    alazar.decimation(1)
    alazar.coupling1('AC')
    alazar.coupling2('AC')
    alazar.channel_range1(0.1)
    alazar.channel_range2(0.1)
    alazar.impedance1(50)
    alazar.impedance2(50)
    alazar.trigger_source1('CHANNEL_A')
    alazar.trigger_level1(128 + 5)
    alazar.external_trigger_coupling('DC')
    alazar.external_trigger_range('ETR_5V')
    alazar.trigger_delay(0)
    alazar.timeout_ticks(int(1e7))
    
    
hetsrc.frequency(5.5668e9)
hetsrc.IF(25e6)
hetsrc.RF.on()
hetsrc.LO.on()
hetsrc.RF.power(-70)
hetsrc.LO.power(15)

S1.power(15)
S1.frequency(4.8e9)
S1.modulation_rf('ON')
S1.pulse_modulation_state('ON')
S1.pulse_modulation_source('EXT')
S1.output_rf('OFF')

def awg_trigger_func(do_start):
    if do_start:
        station.awg.start()
    else:
        station.awg.stop()
        
for ctl in [raw_acq, avgdemod_acq, avgiq_acq]:
    ctl.trigger_func = awg_trigger_func

f:\onedrive\setups\lk2\lk2_code\code\qcodes\qcodes\instrument_drivers\AlazarTech\ATS.py:522: UserWarning: Using Internal clock but parameter external_sample_rate is set.This will have no effect and is ignored
  warnings.warn("Using Internal clock but parameter external_sample_rate is set."


# Spectroscopy

## CW Readout spec

In [50]:
### Spec frequencies
rf_frqs = np.linspace(5.5e9, 5.6e9, 101)[:]


### Setting up instruments

# create new instance of sequence and detector
sequence = create_inst(TriggeredReadoutSequence, 'sequence', awg=awg, force_new_instance=True)
station = add2station(station, sequence)
detector = create_inst(AlazarDetector, name='detector', acqctl=acq, force_new_instance=True)
station = add2station(station, detector)

# Alazar: use IQ detection (one point per trigger)
# let the AWG run freely
acq = avgiq_acq
acq.trigger_func = None

# RF free running
RF.pulsemod_state('off')
RF.on()

# configure the sequence parameters
sequence.pre_trigger_delay(1e-6) # some delay at the beginning
sequence.trigger_len(100e-9) # short trigger for the alazar
sequence.seq_len(10e-6) # quick rep-rate is OK here
sequence.setup(program_awg=False, start_awg=False) # program and start only if something else is running before

# configure the Alazar detector
detector.set_sweeper(sequence) # set the sequence (it's just one dummy sweep-point here)
detector.demod_frq(hetsrc.IF()) # demod frequency from the sources
detector.acq_time(100e-6) # not too long integration time ber buffer
detector.configure_alazar(
    records_per_buffer=1,
    buffers_per_acquisition=400, # set averaging by choosing the number of buffers
)
detector.setup() # this makes sure all params are set correctly

with alazar.syncing(): 
    alazar.trigger_source1('EXTERNAL') # make sure we trigger on the external trigger
    

### Set up the measurement info
exp = select_experiment('readout_spec', SAMPLE) # select experiment (info for the database)

# need to make sure the sweep is correctly described in the beginning!
meas = MeasurementExt(station, exp=exp, 
                      hard_sweep_detector=detector,
                      soft_sweep_params=[
                          hetsrc.frequency,
                      ])

# and finally, the measurement loop
meas.write_period = 2 # how often to write to the DB [s]
with meas.run() as datasaver:
    
    # this is to connect the database to the plottr.
    plot_subscriber = QcodesDatasetSubscriber(datasaver.dataset)
    datasaver.dataset.subscribe(plot_subscriber, state=[])
    
    # the loop itself is trivial
    for f in rf_frqs:
        hetsrc.frequency(f)
        time.sleep(0.01)

        detector.acquisition() # this gets the data from the alazar
        datasaver.add_result(*meas.get_result()) # this adds all data to the datasaver.

Instrument sequence already exists.
Instrument detector already exists.


f:\onedrive\setups\lk2\lk2_code\code\qcodes\qcodes\instrument_drivers\AlazarTech\ATS.py:522: UserWarning: Using Internal clock but parameter external_sample_rate is set.This will have no effect and is ignored
  warnings.warn("Using Internal clock but parameter external_sample_rate is set."


Starting experimental run with id: 20
